# IVR7 Basic Example
This example aims to highlight the follow:
* using time, distance and a single capacity dimension
* locations
* pickup-dropoff tasks (with task times)
* single vehicle class
* single vehicle-cost class
* multiple vehicles

## Requirements
This example assumes you've configured an api-key with the required services enabled (see the [portal](portal.icepack.ai) for configuration details) and that you're familiar with loading and working with protobuf models.

## Data
We're going to load sample data which has order sizes and durations in a tabular format.


In [ ]:
import pandas
df = pandas.read_csv('../sample_data/publist_orders.csv')
print(df.head())

## Model helper
There are several common code-snippets which are useful when working with the IVR series of models, such as creating jobs, defining dimensions etc. 
We've pre-populated examples of such common functions but you're welcome to modify and update them as you see fit for your own application. These wrappers standardise the logic flow across the examples so that we can highlight the differences in functionality between them.

## Model Dimensions
For this example of the IVR7 solver, three dimensions are added - time, distance, and capacity.

For each dimension, the slackMax and tardyMax is set to determine how the dimension behaves. The slackMax provides a limit to how much dimensional quantity may be added in order to perform a task feasibly. In a typical example, like the time dimension, this refers to how much early-waiting time a vehicle may perform for a task. By contrast, the tardyMax provides a limit to how late a task may be relative to its windows. Both the slack and tardy can be costed in the solution to discourage the usage thereof.

For example, when looking at the time dimension the slackMax value indicates how early a vehicle is allowed to arrive before a time window begins while the tardyMax value indicates how long after a time window ends that a vehicle is still permitted complete a task (depending on the value of the tardyMax this may result in a vehicle arriving after the time window closes).

### Dimension - Time
This sets up the time dimension that is used for the model. The built-in units of measurement are:
* seconds (measurementUnit = 0), 
* minutes (measurementUnit = 1),
* hours (measurementUnit = 2), 
* and days (measurementUnit = 3). 
All time is caluclated from a baseline zero hour and will be referenced by vehicles, locations, and tasks. It is up to the user to determine this baseline and then to convert the solution time's to the real-world time equivalent.

In practice, it's easiest to assume that midnight is time zero and everything is measured relative to that. The solver only requires that windows and quantities are specified in the positive axis (so all windows should have values greater than or equal to zero)

### Dimension - Distance
This sets up the distance dimension that is used for the model. The built-in units of measurement are:
* kilometres (measurmentUnit = 4),
* and miles (measurementUnit = 5).

### Dimension - Capacity
This sets up the capacity dimension that is used for the model. This dimension does not have built-in units. This will be referenced by both vehicles and tasks.

In [ ]:
exec(open('apiHelper.py').read()) # import some api-helper classes which we've written for you.
exec(open('ivr7-model-helper.py').read()) # import some modelling helpers


api = apiHelper(modelType="ivr7-kt461v8eoaif") # set the model type to the ivr7 model

sr = ivr7_kt461v8eoaif_pb2.SolveRequest()
sr.solveType = 0 # optimise solve request

m = sr.model

m.dimensions.CopyFrom(make_distance_time_cap_dims())

print(m)


## Locations

We're going to treat the first point as where vehicles are going to begin and end each day. Unlike the tsp/cvrp/pdp models, the ivr7 requires that you specify the unique locations that are going to be used in the model as a separate entity. The reason for this is that you can then specify the locations once, and reference those locations by id for other entities (such and vehicles/jobs/tasks)

In [ ]:
del m.locations[:] # just in case you're running this twice, clear the field before extending it.
m.locations.extend(make_locations(df)) #using the wrapper function

print(m.locations[0])
str(len(m.locations)) + " locations"

## Jobs
For each job, there is a unique ID, a penalty to be applied if the job is not completed as well as associated tasks.

Jobs with higher penalties will be prioritised over jobs with lower penalties.

### Tasks
Jobs also have associated tasks with each task occuring at assigned locations and must be completed by the same vehicle. If one task cannot be completed then the entire job will unperformed in the solution. If a job is unperformed, it will be returned in the Infeasibilites message of the solution along with reasons as to why it could not be completed.

We've constructed some jobs with pickups and dropoffs, loading and offload times, as well as the contribution to the capacity dimension (through the attributes). In this example, all pickup up orders originate at the guiness storehouse and deliver to each customer in the list. `make_job_time_cap` is just a simple function to create this particular style of request, but you can make your own.
 
### Dimension - Capacity
A positive quantity value indicates goods being added to the vehicle and a negative quantity value indicates goods being removed from the vehicle.


In [ ]:
njobs = (df.shape[0])
src = [0] * (njobs-1)
dest =  range(1, njobs)

del m.jobs[:]
m.jobs.extend(make_job_time_cap(df, src, dest))
# lets inspect just the first job created
print(m.jobs[0])

### Vehicle Configuration Vehicle Cost Classes
This is the start of the vehicle-configuration. 
We need to specify:
1. the cost classes available, 
2. the vehicle classes available, 
3. and then the individual vehicles.
we're going to create one of each to keep things simple.

### Vehicle Cost Class

The vehicle cost class describes how the vehicle should be priced for each unit of a dimension assigned to the vehicle class. Multiple cost classes can be created to model differential pricing between vehicle types.

In [ ]:
del m.vehicleCostClasses[:]
m.vehicleCostClasses.append(make_vcc_simple('vcc1', 1000, 1.001e-2, 1.001e-2, 1.001e-2, 1, 3))
print(m.vehicleCostClasses)


### Vehicle Class
A vehicle class describes how the vehicle moves through the network. In other words, we can use the standard network travel speeds, or we could make the vehicle move slower/faster relative to the road network. We could also attach transit rules here which are great for modelling lunch breaks, refueling stops etc. (i.e. conditional triggers on the cumul values of the dimension). Transit rules are covered in advanced examples.


In [ ]:
del m.vehicleClasses[:]
m.vehicleClasses.append(make_vc_simple('vc1', 1, 1, 1, 1))
print("Vehicle Classes")
print(m.vehicleClasses)


### Vehicles
The solver requires a fixed number of vehicle instances to work with. In this model, we can create 2 vehicles all starting and ending their day at the depot with a capacity of 2000 units. 

In [ ]:
del m.vehicles[:]
for i in range(1,2):
    m.vehicles.append( make_vehicle_cap('vehicle_' + str(i), 'vc1', 'vcc1',
                                        2000, # the vehicle capacity
                                       df['id'][0], # the start location
                                       df['id'][0], # the end location
                                       7*60,        # the start time (7AM)
                                       18*60))       # the end time  (6PM)
print("Vehicle 1")
print(m.vehicles[0])

### Sending the model
We now have a complete model specified so we can submit it to the api.

In [ ]:
reqId = api.Post(sr)
print(reqId) # if you want to see what the guid looks like.

### Retrieve the model response
We can use the api-helper to grab the solution response using the request-id provided by the post.

In [ ]:
sol = api.Get(reqId)

print(sol.objective) # This is the cost of the solution returned by the api.


### Tabluating the response

It's quite convenient to table the response data from the api into a collection of stops (nodes) and inter-stops (edges). The reason for this separation between nodes and edges is that they represent different states in the model. Node dimensional quantities refer to quantities incurred while _standing_ and, by contrast, when talking about edges the quantities refer to the _movement_ between tasks. 


In [ ]:
print(sol.routes[0])

In [ ]:
t = tabulate(sr, sol) # pass in the solve request (we grab the locations from the mode) and the solution response
print(t.keys())
print("node table has " + str(t['nodes'].shape[0]) + ' rows and ' + str(t['nodes'].shape[1]) + ' colums' )
print("edge table has " + str(t['edges'].shape[0]) + ' rows and ' + str(t['edges'].shape[1]) + ' colums' )

### Inspecting the tabulated result

Lets start with the stop-table (or node data):


In [ ]:
t['nodes']


Similarly for the edges, we can check the table. Note that the geometries for the edges have been compressed into a list object within each row  (this makes things a touch easier for plotting). 



In [ ]:
t['edges']


Fun thigs to note:
* The capacity start and capacity end in the edge table is always the same. This is because at the start of an action moving a vehicle between two nodes, and at the end of that action, the capacity of the vehicle doesn't change (because no loading of offloading has occured in this example).
* The capacity changes in the node table. You can see this is the cumulative capacity of all orders on the vehicle at each stage in the schedule. This is true for all dimensional values.
* If you take the distance_end - distance_start on the edge table, you'll get the total distance travelled by each vehicle.


### Visualising the response

Lets go ahead and visualise the routes.


In [ ]:
from ipyleaflet import Map, Polyline, Circle, LayerGroup

cvec = list(['blue', 'red', 'orange', 'green', 'black'])
cdict = {}
for i in range(0,5):
    cdict['vehicle_' + str(i)] = cvec[i]


# form some locations to add to the map (just green dots seems ok)
locs = list()
for index, p in t['nodes'].iterrows():
    circle = Circle()
    circle.location = (p['y'], p['x'])
    circle.radius = 10
    circle.color = "green"
    locs.append(circle)

center = [df['Y'][1],df['X'][1]] # just use the first point as the center (i.e. the depot)
m = Map(scroll_wheel_zoom=True, center=center, zoom=12)
for i, gs in enumerate(t['edges']['geometry']):
    m.add_layer(Polyline(locations=gs,color=cdict[t['edges']['vehicleId'][i]],fill=False))

m.add_layer(LayerGroup(layers=(locs)))
m

### What next?
If this example makes sense you can probably proceed to working with more complex modelling activities. See the IVR7 intermediate and advanced examples.